In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup
from time import sleep
import csv
import pandas as pd
import re
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import requests

print('- Finish importing packages')

- Finish importing packages


In [32]:
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
driver = webdriver.Firefox(options=options)
# driver = webdriver.Chrome()
driver.get('https://vnexpress.net/')
sleep(1)
print('- Finish initializing a driver')

- Finish initializing a driver


In [33]:
page_source = BeautifulSoup(driver.page_source)
list_elements = page_source.find_all('h3', class_ = "title-news")
list_links = [e.find('a').get('href') for e in list_elements]

In [38]:
lst_dic = []

for link in list_links:
  
  driver.get(link)
  article_html = BeautifulSoup(driver.page_source)
  try:
    title = article_html.find_all('h1', class_ = 'title-detail')[0].get_text()
  except:
    title = "Unknown"

  try:
    date = article_html.find_all('span', class_ = 'date')[0].get_text()
  except:
    date = ""

  author_list = article_html.find_all('p', class_ = 'author_mail')
  
  try:
    if len(author_list) != 0:
      author = author_list[0].get_text()
    else:
      author = article_html.find_all('h3', class_ = 'title-news')[0].get_text()
  except:
    author = "Unknown"

  try:
    lst_content = [e.get_text() for e in article_html.find('article', class_ = 'fck_detail').findAll('p', class_ = "Normal")]
    content = "\n".join(lst_content)
  except:
    content = ""

  try:
    list_img_links = [e.get('src') for e in article_html.find('article', class_ = 'fck_detail').findAll('img') if e.get('src').startswith("https")]
  except:
    list_img_links = []

  dic = {
    "title": title,
    "date": date,
    "author": author,
    "content": content,
    "imgs": list_img_links
  }

  lst_dic.append(dic)

In [43]:
len(lst_dic)

60

In [49]:
# import urllib
# from pymongo import MongoClient
# client = MongoClient("mongodb+srv://vietthangc1:"+urllib.parse.quote_plus('f2bdx@*-uLAZz!f')+"@cluster0.le7ea.mongodb.net/test")
# db = client.vnexpress

import json
f = open("../app/data/article.json", 'w', encoding='utf-8')
json.dump(lst_dic, f)
f.close()